In [ ]:
import pandas as pd
from dython.nominal import associations

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data = pd.read_pickle('Data/train_data.pkl')
train_labels = pd.read_pickle('Data/train_labels.pkl')

In [ ]:
categorical_features = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [ ]:
association_dictionary = associations(train_data[categorical_features], nominal_columns = categorical_features, mark_columns = True, 
                nom_nom_assoc = 'theil', nan_strategy = 'drop_samples', figsize= (15, 15), vmin = 0, vmax=0.8, compute_only = False)

In [ ]:
print('highest associations:')
tdf = association_dictionary['corr'].stack()
tdf = tdf[tdf.index.get_level_values(0) < tdf.index.get_level_values(1)]
tdf = tdf.sort_values(ascending = False)
print(tdf)

In [ ]:
train_feature_target = pd.concat([train_data.groupby('customer_ID').tail(1), train_labels], axis=1)
train_feature_target.head()

In [ ]:
print('\nThe association between a categorical target and categorical features:')
association_dictionary = associations(train_feature_target[categorical_features + ['target']], 
                                        nominal_columns = categorical_features + ['target'], mark_columns = True, 
                                        display_rows = ['target'], nan_strategy = 'drop_samples',figsize= (15, 15),
                                        vmin = 0, vmax=0.8, compute_only = False, cbar = False) 

In [ ]:
numerical_features = list(set(train_data.columns).difference(set(categorical_features)))
numerical_features.remove('S_2')
train_feature_target = train_feature_target.fillna(train_feature_target.median())

In [ ]:
correlation_dictionary_ff = associations(train_feature_target[numerical_features], numerical_columns = numerical_features, 
                                        mark_columns = False, num_num_assoc = 'pearson', nan_strategy = 'drop_samples',
                                        figsize= (15, 15), vmin = 0, vmax=0.8, compute_only = True)

In [ ]:
print('highest correlations:')
tdf = correlation_dictionary_ff['corr'].stack()
tdf = tdf[tdf.index.get_level_values(0) < tdf.index.get_level_values(1)]
tdf = tdf.sort_values(ascending = False)
# tdf.to_csv('Data/feature_feature_corr.csv')
print(tdf)

In [ ]:
print('\nThe correlation ration (eta) of a categorical target and numerical features:')
correlation_dictionary_ft = associations(train_feature_target[numerical_features + ['target']], nominal_columns = 'target',
                                        numerical_columns = numerical_features, mark_columns = True, 
                                        display_rows = ['target'], nan_strategy = 'drop_samples', 
                                        figsize= (15, 15), vmin = 0, vmax=0.8, compute_only = True, cbar = False)

In [ ]:
cft = correlation_dictionary_ft['corr'].stack().sort_values(ascending = False)
cft.to_csv('Data/feature_target_corr.csv')
cft